# Plasma Beta with FGM and FPI data

This notebook shows how to calculate plasma beta with FGM and FPI data

## Get started

In [1]:
from pyspedas.mms import fgm, fpi # load routines
from pyspedas import tinterpol # interpolate tplot variables
from pytplot import tplot, get_data, store_data # plot, access, and create tplot variables
import numpy as np

## Define some constants

In [2]:
mu0 = 1256.0 # nT-m/A
Kb=1.3807*10**(-16.) # cm^2-g-1/s^2-1/K

## Set some options for the load routines

In [3]:
probe = '1'
trange = ['2015-10-16/11:00', '2015-10-16/14:00']

## Load the FGM and FPI data

In [ ]:
fgm_vars = fgm(trange=trange, probe=probe)
fpi_vars = fpi(datatype=['dis-moms', 'des-moms'], trange=trange, probe=probe)

18-Oct-19 00:22:35: Loading /Volumes/data/data/mms/mms1/fgm/srvy/l2/2015/10/mms1_fgm_srvy_l2_20151016_v4.18.0.cdf


## Set some variable names

In [ ]:
temp_para_i = 'mms'+probe+'_dis_temppara_fast'
temp_perp_i = 'mms'+probe+'_dis_tempperp_fast'
temp_para_e = 'mms'+probe+'_des_temppara_fast'
temp_perp_e = 'mms'+probe+'_des_tempperp_fast'
number_density_i = 'mms'+probe+'_dis_numberdensity_fast'
number_density_e = 'mms'+probe+'_des_numberdensity_fast'
b_field = 'mms'+probe+'_fgm_b_gsm_srvy_l2'
b_magnitude = 'mms'+probe+'_fgm_b_gsm_srvy_l2_mag'

## Split the B-field magnitude into its own variable

In [ ]:
times, data = get_data(b_field)
store_data(b_magnitude, data={'x': times, 'y': data[:, 3]})

## Interpolate the B-field data to the FPI time stamps

In [ ]:
tinterpol(b_magnitude, number_density_i, newname=['b_mag_interpolated'])

## Extract the data from the pyTplot variables

In [ ]:
btimes, bdata = get_data('b_mag_interpolated')

ipatimes, i_para_temp = get_data(temp_para_i)
ipetimes, i_perp_temp = get_data(temp_perp_i)
epatimes, e_para_temp = get_data(temp_para_e)
epetimes, e_perp_temp = get_data(temp_perp_e)

nitimes, i_n = get_data(number_density_i)
netimes, e_n = get_data(number_density_e)

## Calculate the magnetic pressure

note: 1.0e-8 comes from A-nT/m -> g/(s^2-cm)

In [ ]:
Pmag = 1.0e-8*bdata**2/(2.0*mu0)

## Calculate the ion and electron temperatures

In [ ]:
Te_total=(e_para_temp+2*e_perp_temp)/3.0
Ti_total=(i_para_temp+2*i_perp_temp)/3.0

## Calculate the plasma pressure

note: eV -> K conversion: 11604.505 K/eV

In [ ]:
Pplasma = (i_n*11604.505*Ti_total+e_n*11604.505*Te_total)*Kb

## Calculate plasma beta

In [ ]:
Beta = Pplasma/Pmag

## Store the data in pyTplot variables

In [ ]:
store_data('plasma_beta', data={'x': btimes, 'y': Beta})
store_data('magnetic_pressure', data={'x': btimes, 'y': Pmag})
store_data('plasma_pressure', data={'x': btimes, 'y': Pplasma})

## Plot the results

In [ ]:
tplot(['plasma_beta', 'plasma_pressure', 'magnetic_pressure'])